In [35]:
import pandas as pd
import os
import sys
import urllib
import numpy as np

In [36]:
ROOT_DIR = os.path.abspath('.')

QUERY_URL = 'http://www.iwencai.com/data-robot/search?qs=pc_~soniu~stock~resultpage~datarobot~zdjgy&query=%s&isChatBot=0&direct_mode=["iwencai"]'
QUERY_COLUMN = [
    '股票代码', '股票简称', '收盘价不复权', '收盘价前复权', '收盘价后复权', '总市值', '流通市值', '市净率', '市盈率',
    '市销率', '市现率', '成交额', '成交量', '股本', '每股净资产', '净资产收益率', '总资产', '净资产', '负债',
    '货币资金', '营业收入', '净利润', '未分配利润', '营业收入增长率', '资产负债率', '毛利率', '申万行业', '证监会行业',
    '同花顺行业', '上市时间'
]

STATISTIC_COLUMN = [
    '收盘价不复权', '收盘价前复权', '收盘价后复权', '总市值', '流通市值', '市净率', '市盈率', '市销率', '市现率',
    '成交额', '成交量', '股本', '每股净资产', '净资产收益率', '总资产', '净资产', '负债', '货币资金', '营业收入',
    '净利润', '未分配利润', '营业收入增长率', '资产负债率', '毛利率'
]

CLASSICFICATION = [
    '采掘', '传媒', '电气设备', '电子', '房地产', '纺织服装', '非银金融', '钢铁', '公用事业', '国防军工',
    '化工', '机械设备', '计算机', '家用电器', '建筑材料', '建筑装饰', '交通运输', '农林牧渔', '汽车', '轻工制造',
    '商业贸易', '食品饮料', '通信', '休闲服务', '医药生物', '银行', '有色金属', '综合'
]

DATA_DIR = ROOT_DIR + '\\data'
BASIC_DATA_DIR = DATA_DIR + '\\basic'
STOCK_DATA_DIR = DATA_DIR + '\\stock'
TRADE_DATE_FILE = DATA_DIR + '\\date_week.txt'
ALL_STOCK_FILE = DATA_DIR + '\\stock.data'
STATISTIC_FILE = DATA_DIR + '\\statistic.data'
CLASSIFIED_FILE = DATA_DIR + '\\classified.data'
MARKET = ['全部市场', '上海市场', '深圳市场', '创业板市场']
METHOD = ['count', 'std', 'min', '25%', '50%', '75%', 'max', 'mean', 'sum']

# 同花顺问财iwencai数据机器人数据导出

## 数据访问方式

http://www.iwencai.com/data-robot/search?qs=pc_~soniu~stock~resultpage~datarobot~zdjgy&query=%s&isChatBot=0&direct_mode=["iwencai"]

* 这个查询接口曾经出现过一段时间，现在已经在问财网站上找不到公开的调用

关键在于查询问句query，比如 “今年创新低的股票”，“2011.1.1 股价，PB”...

返回的是json格式


## 主要思路

### 用途

用于导出个股历史数据，形成本地的基础数据库，以便于保存 统计 查询 等功能

导出频率以一周为一个数据快照

### 功能
#### get_Snapshot(date,  query)
Parameters：
date   查询快照日期
query  查询快照问句

Return:
返回查询结果dataframe

In [37]:
def get_Snapshot(date, query):
    qstr = pd.Series(query)
    request = QUERY_URL % urllib.parse.quote(qstr.str.cat(sep='[%s],' % date))
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.URLError as e:
        print(e.reason)
    j_data = json.loads(response.read())
    ss = pd.DataFrame(j_data['data']['result'])
    ss.columns = QUERY_COLUMN
    ss = ss.dropna(subset=['上市时间'])
    ss = ss[ss['上市时间'] <= date]
    ss = ss.reset_index(drop=True)
    ss = ss.fillna(0)
    for col in STATISTIC_COLUMN:
        ss[col] = ss[col].astype(float)
    return ss

#### save_Snapshot(snapshot, filename)
用Pickle方式序列化snapshot

In [38]:
def save_Snapshot(snapshot, filename, overwrite=True):
    if os.path.exists(filename) and not overwrite:
        print('已存在该数据文件%s.' % filename)
        return False
    else:
        return snapshot.to_pickle(filename)

#### load_Snapshot(filename)
读取snapshot

Parameters:

filename snapshot文件

Return:

返回dataframe

In [39]:
def load_Snapshot(filename):
    if not os.path.exists(filename):
        print('不存在该数据文件%s.' % filename)
        return False
    else:
        return pd.read_pickle(filename)

In [40]:
def date_to_str(date):
    d = date.split('/', 3)
    return '%04d%02d%02d' % (int(d[0]), int(d[1]), int(d[2]))


def get_TradeDate():
    filename = TRADE_DATE_FILE
    if os.path.exists(filename):
        wd = pd.read_table(filename, encoding='utf-8', header=None)
        return wd
    else:
        print('不存在该日期文件%s.' % filename)
        return False

#### build_local_Snapshot(start_end, query)
查询需要的问句，并保存到本地，形成本地数据库。一个文件代表一天的数据快照。

Parameters：

start_end    查询开始结束时间，是一个日期list

query        查询问句

In [41]:
def build_local_Snapshot(start_end, query):
    print('正在重建本地基础数据库(交易日快照)，耗时比较长，请耐心等候！', end='')

    total = len(start_end)
    i = 0
    # 下载基础数据库，一个date，一个数据文件
    for date in start_end:
        filename = '%s/%s.data' % (BASIC_DATA_DIR, date_to_str(date))
        if not os.path.exists(filename):
            ss = get_Snapshot(date, query)
            save_Snapshot(ss, filename)

        print('[本地基础数据库(交易日快照)] 数据处理中(%d/%d)' % (i, total), end='')
        i += 1
    print('重建本地基础数据库(交易日快照)完毕！', end='')

In [45]:
#build_local_Snapshot(get_TradeDate()[0], QUERY_COLUMN)

#### build_stock_Snapshot()
将下载的每天数据快照，全部整合成一个数据快照，方便做一些中间的计算。

In [116]:
def build_stock_Snapshot():
    print('正在重建整合数据，耗时比较长，请耐心等候！', end='')
    stock = pd.DataFrame()

    for filenames in os.listdir(BASIC_DATA_DIR):
        i = 0
        total = len(filenames)
        # 根据下载的基础数据库，全部整合到一个数据文件
        for filename in filenames:
            fname = os.path.splitext(filename)
            # 格式化时间 yyyy-MM-dd
            date = fname[0]
            ss = load_Snapshot('%s/%s' % (BASIC_DATA_DIR, filename))
            ss['date'] = date
            stock = stock.append(pd.DataFrame(ss))
            print('[整合数据] 数据处理中(%d/%d)' % (i, total), end='')
            i += 1
    # NaN 用 0 填充
    stock = stock.fillna(0)
    save_Snapshot(stock, ALL_STOCK_FILE)
    print('重建本地整合数据完毕！', end='')

#### split_stock_Snapshot()
把整合全部的数据快照，按照股票代码切割出来。形成一个文件是一个股票历史数据快照。

In [22]:
def split_stock_Snapshot():
    print('正在重建本地基础数据库(个股快照)，耗时比较长，请耐心等候！', end='')
    if not os.path.exists(ALL_STOCK_FILE):
        build_stock_Snapshot()
    else:
        stock = load_Snapshot(ALL_STOCK_FILE)
        stk = stock.drop_duplicates(subset=['股票代码'], keep='first')
        i = 0
        total = len(stk)
        # 将切割成一个股票，一个历史文件
        for code in stk['股票代码']:
            s = stock[stock['股票代码'].str.contains(code)]
            s = s.reset_index(drop=True)

            filename = '%s/%s.data' % (STOCK_DATA_DIR, code.split('.', 2)[0])
            save_Snapshot(s, filename)
            print('[本地基础数据库(个股快照)] 数据处理中(%d/%d)' % (i, total), end='')
            i += 1
    print('重建本地基础数据库(个股快照)完毕！', end='')

#### build_statistic_Snapshot()
对每天的数据快照做一个统计计算，并合并成一个统计数据快照。

In [115]:
def build_statistic_Snapshot():
    print('正在重建个股统计数据，耗时比较长，请耐心等候！', end='')
    statistic = pd.DataFrame()

    for filenames in os.listdir(BASIC_DATA_DIR):
        i = 0
        total = len(filenames)
        for filename in filenames:
            fname = os.path.splitext(filename)
            # 格式化时间 yyyy-MM-dd
            date = fname[0]
            ss = load_Snapshot('%s/%s' % (BASIC_DATA_DIR, filename))

            # 全市场统计
            desc = ss.describe()

            desc.loc['sum'] = ss.sum()
            desc['date'] = date
            desc['market'] = '全部市场'
            statistic = statistic.append(pd.DataFrame(desc))

            # 上海市场统计
            dt = ss[ss['股票代码'].str.contains(r'^6')]
            desc = dt.describe()
            desc.loc['sum'] = dt.sum()
            desc['date'] = date
            desc['market'] = '上海市场'
            statistic = statistic.append(pd.DataFrame(desc))

            # 深圳市场统计
            dt = ss[ss['股票代码'].str.contains(r'^0')]
            desc = dt.describe()
            desc.loc['sum'] = dt.sum()
            desc['date'] = date
            desc['market'] = '深圳市场'
            statistic = statistic.append(pd.DataFrame(desc))

            # 创业板市场统计
            dt = ss[ss['股票代码'].str.contains(r'^3')]
            desc = dt.describe()
            desc.loc['sum'] = dt.sum()
            desc['date'] = date
            desc['market'] = '创业板市场'
            statistic = statistic.append(pd.DataFrame(desc))
            print('[个股统计] 数据处理中(%d/%d)' % (i, total), end='')
            i += 1
    # NaN 用 0 填充
    statistic = statistic.fillna(0)
    save_Snapshot(statistic, STATISTIC_FILE)
    print('重建个股统计数据完毕！', end='')

#### build_classified_Snapshot()

In [114]:
def build_classified_Snapshot():
    print('正在重建行业统计数据，耗时比较长，请耐心等候！', end='')
    statistic = pd.DataFrame()
    for filenames in os.listdir(BASIC_DATA_DIR):
        i = 0
        total = len(filenames)

        for filename in filenames:
            fname = os.path.splitext(filename)
            # 格式化时间 yyyy-MM-dd
            date = fname[0]
            ss = load_Snapshot('%s/%s' % (BASIC_DATA_DIR, filename))

            # Groupby 行业
            ss['行业'] = ss['申万行业'].apply(lambda x: x.split('-')[0])
            ss['date'] = date
            dt = ss.groupby(['date', '行业'])
            desc = dt.agg([
                'count', 'std', 'min', ('25%',
                                        lambda x: pd.Series.quantile(x, .25)),
                ('50%', lambda x: pd.Series.quantile(x, .5)),
                ('75%',
                 lambda x: pd.Series.quantile(x, .75)), 'max', 'mean', 'sum'
            ])
            desc = desc.reset_index()
            statistic = statistic.append(pd.DataFrame(desc))
            print('[行业统计] 数据处理中(%d/%d)' % (i, total), end='')
            i += 1
    # NaN 用 0 填充
    statistic = statistic.fillna(0)
    save_Snapshot(statistic, CLASSIFIED_FILE)
    print('重建行业统计数据完毕！', end='')

In [31]:
#build_classified_Snapshot()
#build_statistic_Snapshot()